열일곱번째 제출 23.11.29
###### 전처리 : /255
###### pca : 256
###### svm (c = 16)
###### 이미지 증강(좌, 우 쉬프트 >>  180000)
###### 쉬프트 정도 조정 (0.10)

###### ++ 부류 6 추가학습 svc 합치기


결과: 81.55


In [78]:
# scikit-learn
!pip install scikit-learn

# lightgbm
!pip install lightgbm

# xgboost
!pip install xgboost

# OpenCV
!pip install opencv-python

# tensorflow (이미지 데이터 제너레이션을 사용하기 위해)
!pip install tensorflow

# tqdm
!pip install tqdm

# matplotlib
!pip install matplotlib


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [79]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.decomposition import PCA
import lightgbm as lgb
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier

import os
from tqdm import tqdm
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

In [80]:
training_data = pd.read_csv('/home/user/다운로드/test/fashion-mnist_train.csv')

train_y = training_data['label']

train_X = training_data.drop('label',axis=1)

# # 데이터에서 한 줄을 뽑아와서 28x28 크기의 이미지로 변환
# sample_image = train_X.iloc[0].values.reshape(28, 28)

# # 시각화
# plt.imshow(sample_image, cmap='gray')
# plt.title(f"Label: {train_y.iloc[0]}")
# plt.show()

In [81]:
datagen = ImageDataGenerator(
    width_shift_range=0.10,
    height_shift_range=0.10
)

np.random.seed(42)

# Data augmentation and adding to the training set loop
aug_train_X = []
aug_train_y = []

for index, row in tqdm(train_X.iterrows(), total=len(train_X)):
    random_num = np.random.random()

    img = row.values.reshape((28, 28, 1))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Original data
    aug_train_X.append(img_array[0])
    aug_train_y.append(train_y[index])

    # Augmented data with 2/3 probability
    
    augmented_img_array = next(datagen.flow(img_array, batch_size=1))
    augmented_img_array = augmented_img_array.squeeze(axis=0)
    aug_train_X.append(augmented_img_array)
    aug_train_y.append(train_y[index])

print(len(aug_train_X))
print(len(aug_train_y))

100%|██████████| 60000/60000 [00:09<00:00, 6475.07it/s]

120000
120000


In [82]:
for index, row in tqdm(train_X.iterrows(), total=len(train_X)):
    random_num = np.random.random()

    img = row.values.reshape((28, 28, 1))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    # Original data
    # aug_train_X.append(img_array[0])
    # aug_train_y.append(train_y[index])

    augmented_img_array = next(datagen.flow(img_array, batch_size=1))
    augmented_img_array = augmented_img_array.squeeze(axis=0)
    aug_train_X.append(augmented_img_array)
    aug_train_y.append(train_y[index])
    
# Convert the lists to numpy arrays
aug_train_X = np.array(aug_train_X)
aug_train_y = np.array(aug_train_y)

# Check the shape of the new arrays
print("Shape of aug_train_X:", aug_train_X.shape)
print("Shape of aug_train_y:", aug_train_y.shape)

100%|██████████| 60000/60000 [00:09<00:00, 6505.85it/s]


Shape of aug_train_X: (180000, 28, 28, 1)
Shape of aug_train_y: (180000,)


In [83]:
aug_train_X = aug_train_X.reshape((aug_train_X.shape[0], -1))
aug_train_X/=255.0

In [84]:
print(aug_train_X.shape)
# aug_train_X[2]

(180000, 784)


In [85]:
pca = PCA( n_components= 256 )
aug_train_X = pca.fit_transform(aug_train_X)

# 앙상블 테스트 LGB



In [86]:
# LightGBM 모델 초기화 및 학습


model = lgb.LGBMClassifier()
model.fit(aug_train_X, aug_train_y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.102619 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 65280
[LightGBM] [Info] Number of data points in the train set: 180000, number of used features: 256
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585
[LightGBM] [Info] Start training from score -2.302585


LGBMClassifier()

In [87]:
test_data_folder = "/home/user/다운로드/test/public_test_dataset/data"

# 테스트 데이터 로드 및 전처리
test_X = []
file_names = []

print(len(os.listdir(test_data_folder)))

# tqdm으로 래핑
for file_name in tqdm(os.listdir(test_data_folder), desc="Loading and preprocessing"):
        if file_name.endswith(".png"):
            file_path = os.path.join(test_data_folder, file_name)
            try:
                image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                image_array = image / 255.0  # 0부터 1사이의 값
                test_X.append(image_array.flatten())  # 2D 배열을 1D로 펼침
                # print(image_array)

                file_names.append(file_name)
            except Exception as e:
                print(f"Error processing file {file_name}: {str(e)}")


# NumPy 배열로 변환
test_X = np.array(test_X)
print(test_X.shape)

if len(test_X) == 0:
    print("No valid test images found.")
else:
    print("pca 적용")
    test_X = pca.transform(test_X)
    print(test_X.shape)

    # 테스트 데이터에 대한 예측
    final_predictions = model.predict(test_X)


with open('/home/user/다운로드/test/testResultLgb.txt', 'w') as file:
    for i, pred in enumerate(final_predictions):
        file.write(f"{i:05d} {pred}\n")

10000


Loading and preprocessing: 100%|██████████| 10000/10000 [00:00<00:00, 49004.15it/s]


(10000, 784)
pca 적용
(10000, 256)


In [88]:
import sys
import numpy as np
from sklearn.metrics import auc
from collections import Counter

testResult_path = "/home/user/다운로드/test/testResultLgb.txt"
label_path = "/home/user/다운로드/test/public_test_dataset/label.txt"

# pred에 해당하는 testResult.txt 파일 읽어오는 부분입니다.
with open(testResult_path, 'r') as file1:
    preds = file1.readlines()

# 정답에 해당하는 label.txt 파일 읽어오는 부분입니다.
with open(label_path, 'r') as file2:
    labels = file2.readlines()


# pred와 label의 클래스값만 리스트로 변환하는 부분입니다.
p = np.array([pred.strip().split()[1] for pred in preds])
l = np.array([label.strip().split()[1] for label in labels])

# pred의 클래스 개수를 count하는 부분입니다.
predict_label_count_dict = Counter(p)
predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

## mAP 계산하는 부분입니다.
AP = []
num_class = 10

# 모든 클래스에 대해 반복
for c, freq in predict_label_count_dict.items() :
    TP = 0
    FN = 0

    temp_precision = []
    temp_recall = []

    for i in range(len(p)):
        # TP, FN 계산
        if l[i] == c and p[i] == c :
            TP += 1
        elif l[i] != c and p[i] == c :
            FN += 1

        # preciison, recall 계산
        if TP+FN != 0:
            temp_precision.append(TP/(TP+FN))
            temp_recall.append(TP/freq)

    # AP 배열에 클래스 각각의 AP value 저장
    # auc : preciison-recall curve의 면적 구해줌
    AP.append(auc(temp_recall, temp_precision))

mAP = sum(AP) / num_class

# 각각의 클래스에 대한 AP와 mAP의 Table 출력 부분입니다.
class_name = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
table = "| {:<13} | {:<13} |\n".format("Class", "AP") + "|---------------|---------------|\n"

for c_name, ap in zip(class_name, AP):
    table += "| {:<13} | {:<13.3f} |\n".format(c_name, ap)

table += "| {:<13} | {:<13.3f} |\n".format("mAP", mAP)

print(table)

| Class         | AP            |
|---------------|---------------|
| T-shirt/top   | 0.012         |
| Trouser       | 0.008         |
| Pullover      | 0.008         |
| Dress         | 0.014         |
| Coat          | 0.013         |
| Sandal        | 0.009         |
| Shirt         | 0.012         |
| Sneaker       | 0.009         |
| Bag           | 0.010         |
| Ankle boot    | 0.008         |
| mAP           | 0.010         |



In [89]:

# with open('../testResult17(다반1조).txt', 'w') as file:
#     for i, pred in enumerate(final_predictions):
#         file.write(f"{i:05d} {pred}\n")

In [90]:
# for i in range(10):
#     print(final_predictions[i])